In [5]:
# Install all packages
from tkinter import *
import tkinter as tk
import copy
import numpy as np
from datetime import datetime
from enum import Enum

In [6]:
class Result(Enum):
    """ A class that encodes the result of the game."""
    X = 1
    O = -1
    DRAW = 0   
    
    def __repr__(self):
        return str(self.name)

class Player(Enum):
    """A class that encodes players and switches to opponents easily."""
    X = 1
    O = 0
    EMPTY = -1 
    
    def switch(self):
        """ Funcion that perform the switch"""
        return Player(not self.value)
    
    def __repr__(self):
        return str(self.name[0] if self.name[0] != 'E' else '.')
    
    def __lt__(self, other):
        """ Overwrite the less then function to compare values easily"""
        return self.value < other.value 
    

In [7]:
class TicTacToe:
    """ A class that implements a generalized version of a tic-tac-toe
    game with the help of the minimax algorithm.
    """
    def __init__(self, n=3, cons_tiles = 3):
        """ Initialize an instance of a class.
        Arguments:
        n is the size of the side of the board. 
        cons_tiles is the number of consequitive tiles that
        players need to mark.
        """
        self.board = np.repeat(Player.EMPTY, n ** 2).reshape(n, n)
        self.x_wins = 0
        self.o_wins = 0
        self.n = n
        self.cons_tiles = cons_tiles
        self.last_move = None
        
    def printing(self):
        """ Prints out the game board."""
        print (self.board)
        
    def ok(self, r, c):
        """ Checks whether the row and column values are inside the board."""
        return (r >= 0 and r < self.n) and (c >= 0 and c < self.n)

    def game_over(self): 
        """ A function that identifies whether the game has finished.
        It returns "X", "Y", or "DRAW" as well as the winning tiles."""
        # Check all directions
        dx = [0,  0, 1, -1, 1, -1, 1, -1]
        dy = [1, -1, 0,  0, 1, -1, -1, 1]

        r, c = self.last_move
        same_in_dir = []
        # Iterate in every direction from the last cell
        for direction in range(len(dx)):
            same_in_dir.append(0)
            for k in range(1, self.cons_tiles):
                new_r = r + dx[direction] * k
                new_c = c + dy[direction] * k
                if not self.ok(new_r, new_c):
                    break
                if self.board[new_r][new_c] != self.board[r][c]:
                    break
                same_in_dir[-1] += 1
        for x in range(0, len(same_in_dir), 2):
            if same_in_dir[x] + same_in_dir[x + 1] + 1 >= self.cons_tiles:
                start = (r + dx[x] * same_in_dir[x], c + dy[x] * same_in_dir[x])
                line = [start] 
                for i in range(1, self.cons_tiles):
                    line.append((start[0] + dx[x + 1] * i, start[1] + dy[x + 1] * i))
                return Result[Player(self.board[r][c]).name], line
        # If board is not full, return Nones
        if Player.EMPTY in set(np.unique(self.board)):
            return None, []
        # If the board is full and nobody won, return draw
        return Result.DRAW, []                
                
    
    def minimax(self, player, depth=0,alpha = -1000, beta = 1000):
        """ The minimax algorithm with alpha-beta pruning and the depth heuristic."""
        winner, _ = self.game_over()
        if winner is not None:
            return (Result[winner.name], None, None, depth)
        # Set initial outcome values
        outcome = 100 if player is Player.O else -100

        best_r, best_c = None, None
        # Recursively iterate through the squares
        for r in range(self.n):
            for c in range(self.n):
                if self.board[r][c] is Player.EMPTY:
                    self.board[r][c] = player
                    self.last_move = (r, c)
                    res, _, _, dep = self.minimax(player.switch(), depth + 1, alpha, beta)
                    self.board[r][c] = Player.EMPTY
                    # Update outcomes
                    if (player is Player.X and res.value > outcome) or (player is Player.O and res.value < outcome):
                        outcome = res.value    
                        best_r = r
                        best_c = c
                        # OPtimization 1. Alpha-beta pruning
                        if player is Player.X: 
                            alpha = max(alpha, outcome)
                        else: 
                            beta = min(beta, outcome)
                    if alpha >= beta:
                        return Result(outcome), best_r, best_c, depth
                        
                    # Optimization 2: Return if we found a favorable outcome already  
                    if player is Player.X and outcome == 1:
                        return Result(outcome), best_r, best_c, depth
                    elif player is Player.O and outcome == -1:
                        return Result(outcome), best_r, best_c, depth

        return Result(outcome), best_r, best_c, depth
        
    def make_move(self, player, move):
        """ This function marks the squares and determines winners."""
        self.board[move[0], move[1]] = player
        self.last_move = move
        # Check if the game is over
        winner, squares = self.game_over()
        if winner is Result.O:
            self.o_wins += 1
        elif winner is Result.X:
            self.x_wins += 1
        if winner:
            return False
        return True
    
    def reset(self):
        """ Reset the board to start from the beginning."""
        self.board = np.repeat(Player.EMPTY, self.n ** 2).reshape(self.n, self.n)
        self.winning_squares = []
        self.moves = []
        print("Board Reset")

In [8]:
class TicTacToeGUI:
    """ This class implements a graphical user interface for the tic-tac-toe."""
    
    def __init__(self, root, n=3, cons_tiles = 3):
        """ Initialize the instance of a class with a tkinter root,
        number of consecutive tiles required to win and the size of the board."""
        self.n = n
        self.root = root
        self.root.title("CS152, Final Project")
        # Creates a game
        self.game = TicTacToe(n, cons_tiles)
        # Welcome Label
        self.welcome, self.welcome_text = \
            self.create_label("Tic-Tac-Toe", 0, 0, self.n)

        # Display current scores
        self.count, self.count_text = self.create_label("", 1, 0, self.n)
        self.update_win_counter()

        # Button for resetting the game
        self.restart_button, self.restart_button_text = \
            self.create_button("Restart", self.resetGUI, 1, 0, 1)

        # Information about the state of the game
        self.info, self.info_text = self.create_label("", 2, 0, self.n)

        # Set appropriate column and row sizes
        for i in range(self.n):
            self.root.columnconfigure(i, minsize=100)
            self.root.rowconfigure(i + 3, minsize=100)

        # Create buttons
        self.string_board = [StringVar() for _ in range(n**2)]
        for s in self.string_board:
            s.set(" ")
        self.buttons = []
        for r in range(n):
            for c in range(n):
                btn = Button(self.root, 
                             textvariable=self.string_board[r * n + c], 
                             command=lambda gui=self, r=r, c=c: self.button_click(r, c))
                btn.grid(row=r+3, column=c, sticky=NSEW)
                self.buttons.append(btn)
        # Start the GUI loop
        self.root.mainloop()
        
    def create_label(self, text, row, column, span):
        """ Create tkinter labels."""
        string = StringVar()
        string.set(text)
        label = Label(self.root, textvariable=string)
        label.grid(row=row, column=column, columnspan=span)
        return label, string
    
    def create_button(self, text, command, row, column, span):
        """ A subroutine that helps to generate buttons."""
        string = StringVar()
        string.set(text)
        btn = Button(self.root, 
                     textvariable=string, 
                     command=command)
        btn.grid(row=row, column=column, columnspan=span)
        return btn, string 
        
    def button_click(self, r, c):
        """ The function is reponsible for two moves: one by the player, one by the AI. """
        # Player made a move
        self.move(Player.X, (r, c))
        
        # Calculate AI's move
        # Check the running time of the minimax
        start = datetime.now()
        r, moveX, moveY, _ = self.game.minimax(Player.O)
        print (datetime.now()-start)
        
        if moveX is not None:
            # Make AIs move
            self.move(Player.O, (moveX, moveY))
        
    def move(self, player, move):
        """ Responsible for the move"""
        coord = move[0] * self.n + move[1]
        self.string_board[coord].set(player.name)
        self.buttons[coord].config(state="disabled")
        res = self.game.make_move(player, move)
        if not res:
            # The game is over 
            winner, line = self.game.game_over()
            info_text = winner.name + " won!"
            winning_line = [self.buttons[s[0] * self.n + s[1]] for s in line]
            
            if winner == Result.DRAW:
                winning_line = self.buttons
                info_text = "It's a draw!"
                
            # Update the GUI
            self.info_text.set(info_text)
            
            # Color winning line in red 
            for button in winning_line:
                button.config(disabledforeground="red")
            for button in self.buttons:
                button.config(state="disabled")
            self.update_win_counter()
            return
        
    def update_win_counter(self):
        """ Updates the counter after the game is over."""
        x_wins = str(self.game.x_wins)
        o_wins = str(self.game.o_wins)
        self.count_text.set("X: {} \tO: {}".format(x_wins, o_wins))
            
    def resetGUI(self):
        """ A reset function analagous to the one in the tic-tac-toe class."""
        self.info_text.set(" ")
        self.game.reset()
        for s in self.string_board:
            s.set(" ")
            
        for button in self.buttons:
            button.config(state="normal")
            
        for button in self.buttons:
            button.config(disabledforeground="black")
    

In [ ]:
#Play the standard game
root = tk.Tk()
game = TicTacToeGUI(root, 3, 3)

0:00:00.232730
0:00:00.009004


In [ ]:
# Play a 4x4 game with 3 consecutive tiles
root = tk.Tk()
game = TicTacToeGUI(root, 4, 3)

In [ ]:
# Play a 4x4 game with 4 consecutive tiles
# This one is too slow on my computer
root = tk.Tk()
game = TicTacToeGUI(root, 4, 4)

In [4]:
a = "hello"
b = "hello"
print (id(a), id(b))
a == b
id(1.1)==id(0.1)  

4396413592 4396413592


False